## [Process](https://huggingface.co/docs/datasets/process)

In [34]:
from datasets import load_dataset

dataset = load_dataset("data/animal") # ちなみに、`drop_label` の値にかかわらず、`metadata.csv`がある場合は未記載のファイルがあるとエラーになる。

In [35]:
dataset["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=652x515>,
 'label': 'dog'}

In [37]:
dataset["train"][0:]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=652x515>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=660x660>],
 'label': ['dog', ' dog']}

In [38]:
from torchvision import transforms

# 辞書の値がリストであることに注意。
# dataset["train"][0] で呼び出した場合は、len(batch["image"]) == 1 となる。
# dataset["train"][0:] で呼び出した場合は、len(batch["image"]) は元配列の長さとなりそう。 
def transform(batch: dict[str, list[any]]) -> dict[str, list[any]]:
    cropped = [transforms.functional.center_crop(image, 100) for image in batch["image"]]
    return {"image": cropped, "label": batch["label"]}

dataset.set_transform(transform, ["image", "label"])

In [39]:
dataset["train"][0:]

{'image': [<PIL.Image.Image image mode=RGB size=100x100>,
  <PIL.Image.Image image mode=RGB size=100x100>],
 'label': ['dog', ' dog']}